# Mesh for Entire Domain

In [1]:
%reset -f
import h5py,os
import numpy as np

#modify user inputs below
mesh_prefix='sphere'
outfile='sphere_s61.hdf5'
freq=[0.01,0.1] #simulating frequencies

rel_perm_electrolyte=78.5 #relative permittivity of bulk electrolyte [F/m]
rel_perm_solid=4.5 #relative permittivity of solid particle [F/m]
diffusion_electrolyte=2*1e-6**2/1e-3 #diffusion coefficient of water at 20 deg. C is 2*(1e-6*m)^2/(1e-3*s)
diffusion_stern=2*1e-6**2/1e-2 #diffusion coefficient in Stern layer [m^2/s]
mobility_electrolyte=5e-8 #mobility of sodium cation in bulk electrolyte [m^2/(Vs)]
mobility_stern=5e-9 #mobility in Stern layer [m^2/(Vs)]
sigma_electrolyte=-0.01 #surface charge density in electrolyte [C/m^2]
sigma_stern=-0.01 #surface charge density in Stern layer [C/m^2]

temp_K=298 #room/ambient temperature [K]
cinf=100 #ion concentration in bulk electrolyte [mol/m^3]
zval=1 #ion valence in bulk electrolyte
# cinf_pos=0.01 #ion concentration of positive charges
# cinf_neg=0.01 #ion concentration of negative charges
# zval_pos=1 #positive ion valence
# zval_neg=1 #negative ion valence

#set physical constants
echarge=1.602e-19 #elementary charge [C]
perm0=8.85e-12 #vacuum permittivity [F/m]
kA=6.022e23 #Avogadro constant [1/mol]
kB=1.381e-23 #Boltzmann's constant [J/K]

#load mesh
print('Reading %s.1.node'%mesh_prefix)
nodes=np.genfromtxt(mesh_prefix+'.1.node',skip_header=1,skip_footer=1,usecols=(1,2,3))
node_flags=np.genfromtxt(mesh_prefix+'.1.node',skip_header=1,skip_footer=1,usecols=5,dtype='int')

print('Reading %s.1.ele'%mesh_prefix)
elements=np.genfromtxt(mesh_prefix+'.1.ele',skip_header=1,usecols=(1,2,3,4),dtype='int')
zones=np.genfromtxt(mesh_prefix+'.1.ele',skip_header=1,usecols=5,dtype='int')

print('Reading %s.1.face'%mesh_prefix)
faces=np.genfromtxt(mesh_prefix+'.1.face',skip_header=1,usecols=(1,2,3),dtype='int')
face_flags=np.genfromtxt(mesh_prefix+'.1.face',skip_header=1,usecols=4,dtype='int')

#adjust indices to start from zero
elements=elements-1
faces=faces-1

# #translate z coordinate
# print('Translating Z coordinate')
# nodes[:,2]=-(np.power(10,-nodes[:,2]/1000*3)-1) #problem dependent
#adjust mesh by radius
print('Adjusting mesh by radius (will be removed in the future)')
dist1=np.sqrt(nodes[:,0]**2+nodes[:,1]**2+nodes[:,2]**2)
mask=dist1>5.0
dist2=np.zeros_like(dist1)
dist2[mask]=10**((dist1[mask]-5)/50*2)+5-1

# print(min(dist1[mask]),max(dist1[mask]))
# print(min(dist2[mask]),max(dist2[mask]))
nodes[mask,0]=nodes[mask,0]/dist1[mask]*dist2[mask]
nodes[mask,1]=nodes[mask,1]/dist1[mask]*dist2[mask]
nodes[mask,2]=nodes[mask,2]/dist1[mask]*dist2[mask]
#scale nodes from meter to nano-meter
nodes=nodes*1e-9

nnode=len(nodes)
nelem=len(elements)
nface=len(faces)
print('THE NUMBER OF NODES IS: %d'%nnode)
print('THE NUMBER OF ELEMENTS IS: %d'%nelem)
print('THE NUMBER OF FACES IS: %d'%nface)
print('')

#define computational volume and boundaries
in_active_zones=zones>=0 #True for elements used in computation
with_first_kind_bc=(node_flags==1)|(node_flags==2) #True for nodes on first kind b.c.
with_third_kind_bc=face_flags==20 #True for faces on third kind b.c.

#define electrolyte,solid,stern layer
in_electrolyte=zones>=3
in_solid=zones<=2
in_stern=face_flags==20

# print('Compute distance scaling factor using Debye length')
# debye_len=np.sqrt(perm0*rel_perm_electrolyte*kB*temp_K/2/zval**2/echarge**2/kA/cinf)
# dist_factor=10**(-round(np.log10(debye_len))) #to be determined from debye length
# print('DEBYE LENGTH IS: %.2e m'%debye_len)
# print('DISTANCE SCALING FACTOR IS: %.2e'%dist_factor)
# print('')

print('Saving coefficients to %s'%outfile)
with h5py.File(outfile,'w') as file:
    #group mesh
    file.create_dataset('mesh/nodes',data=nodes,dtype='float')
    file.create_dataset('mesh/elements',data=elements,dtype='int')
    file.create_dataset('mesh/faces',data=faces,dtype='int')
            
    file.create_dataset('mesh/in_active_zones',data=in_active_zones,dtype='bool')
    file.create_dataset('mesh/with_first_kind_bc',data=with_first_kind_bc,dtype='bool')
    file.create_dataset('mesh/with_third_kind_bc',data=with_third_kind_bc,dtype='bool')
    
    file.create_dataset('mesh/in_electrolyte',data=in_electrolyte,dtype='bool')
    file.create_dataset('mesh/in_solid',data=in_solid,dtype='bool')
    file.create_dataset('mesh/in_stern',data=in_stern,dtype='bool')
    #file.create_dataset('mesh/dist_factor',data=dist_factor,dtype='float')
    
    #group physics
    file.create_dataset('physics/rel_perm_electrolyte',data=rel_perm_electrolyte,dtype='float')
    file.create_dataset('physics/rel_perm_solid',data=rel_perm_solid,dtype='float')
    file.create_dataset('physics/diffusion_electrolyte',data=diffusion_electrolyte,dtype='float')
    file.create_dataset('physics/diffusion_stern',data=diffusion_stern,dtype='float')
    file.create_dataset('physics/mobility_electrolyte',data=mobility_electrolyte,dtype='float')
    file.create_dataset('physics/mobility_stern',data=mobility_stern,dtype='float')
    file.create_dataset('physics/sigma_electrolyte',data=sigma_electrolyte,dtype='float')
    file.create_dataset('physics/sigma_stern',data=sigma_stern,dtype='float')
    
    file.create_dataset('physics/temperature',data=temp_K,dtype='float')
    file.create_dataset('physics/ion_conc_inf',data=cinf,dtype='float')
    file.create_dataset('physics/ion_val',data=zval,dtype='float')
    
    #create subgroups for frequencies
    for i in range(len(freq)):
        file.create_group('frequency_%.6e'%freq[i])
    
print('Done')

Reading sphere.1.node
Reading sphere.1.ele
Reading sphere.1.face
Adjusting mesh by radius (will be removed in the future)
THE NUMBER OF NODES IS: 24157
THE NUMBER OF ELEMENTS IS: 135851
THE NUMBER OF FACES IS: 17578

Saving coefficients to sphere_s61.hdf5
Done


In [2]:
import h5py
import numpy as np

f=h5py.File('sphere_s61.hdf5','r')
print(f.keys())
print(f['mesh'].keys())
print(f['physics'].keys())
# tmp=np.array(f.get('mesh/elements'))
# print(f['physics/sigma_solid'])
f.close()

<KeysViewHDF5 ['frequency_1.000000e-01', 'frequency_1.000000e-02', 'mesh', 'physics']>
<KeysViewHDF5 ['elements', 'faces', 'in_active_zones', 'in_electrolyte', 'in_solid', 'in_stern', 'nodes', 'with_first_kind_bc', 'with_third_kind_bc']>
<KeysViewHDF5 ['diffusion_electrolyte', 'diffusion_stern', 'ion_conc_inf', 'ion_val', 'mobility_electrolyte', 'mobility_stern', 'rel_perm_electrolyte', 'rel_perm_solid', 'sigma_ss', 'temperature']>


# Mesh for Electrolyte

In [2]:
%reset -f
import h5py,os
import numpy as np

#modify user inputs below
mesh_prefix='sphere'
outfile='sphere_s61_electrolyte.hdf5'
freq=[0.01,0.1] #simulating frequencies

rel_perm_electrolyte=78.5 #relative permittivity of bulk electrolyte [F/m]
rel_perm_solid=4.5 #relative permittivity of solid particle [F/m]
diffusion_electrolyte=2*1e-6**2/1e-3 #diffusion coefficient of water at 20 deg. C is 2*(1e-6*m)^2/(1e-3*s)
diffusion_stern=2*1e-6**2/1e-2 #diffusion coefficient in Stern layer [m^2/s]
mobility_electrolyte=5e-8 #mobility of sodium cation in bulk electrolyte [m^2/(Vs)]
mobility_stern=5e-9 #mobility in Stern layer [m^2/(Vs)]
sigma_electrolyte=-0.01 #surface charge density in electrolyte [C/m^2]
sigma_stern=-0.01 #surface charge density in Stern layer [C/m^2]

temp_K=298 #room/ambient temperature [K]
cinf=100 #ion concentration in bulk electrolyte [mol/m^3]
zval=1 #ion valence in bulk electrolyte
# cinf_pos=0.01 #ion concentration of positive charges
# cinf_neg=0.01 #ion concentration of negative charges
# zval_pos=1 #positive ion valence
# zval_neg=1 #negative ion valence

#set physical constants
echarge=1.602e-19 #elementary charge [C]
perm0=8.85e-12 #vacuum permittivity [F/m]
kA=6.022e23 #Avogadro constant [1/mol]
kB=1.381e-23 #Boltzmann's constant [J/K]

#load mesh
print('Reading %s.1.node'%mesh_prefix)
nodes=np.genfromtxt(mesh_prefix+'.1.node',skip_header=1,skip_footer=1,usecols=(1,2,3))
node_flags=np.genfromtxt(mesh_prefix+'.1.node',skip_header=1,skip_footer=1,usecols=5,dtype='int')

print('Reading %s.1.ele'%mesh_prefix)
elements=np.genfromtxt(mesh_prefix+'.1.ele',skip_header=1,usecols=(1,2,3,4),dtype='int')
zones=np.genfromtxt(mesh_prefix+'.1.ele',skip_header=1,usecols=5,dtype='int')

print('Reading %s.1.face'%mesh_prefix)
faces=np.genfromtxt(mesh_prefix+'.1.face',skip_header=1,usecols=(1,2,3),dtype='int')
face_flags=np.genfromtxt(mesh_prefix+'.1.face',skip_header=1,usecols=4,dtype='int')

#adjust indices to start from zero
elements=elements-1
faces=faces-1

# #translate z coordinate
# print('Translating Z coordinate')
# nodes[:,2]=-(np.power(10,-nodes[:,2]/1000*3)-1) #problem dependent
#adjust mesh by radius
print('Adjusting mesh by radius (will be removed in the future)')
dist1=np.sqrt(nodes[:,0]**2+nodes[:,1]**2+nodes[:,2]**2)
mask=dist1>5.0
dist2=np.zeros_like(dist1)
dist2[mask]=10**((dist1[mask]-5)/50*2)+5-1

# print(min(dist1[mask]),max(dist1[mask]))
# print(min(dist2[mask]),max(dist2[mask]))
nodes[mask,0]=nodes[mask,0]/dist1[mask]*dist2[mask]
nodes[mask,1]=nodes[mask,1]/dist1[mask]*dist2[mask]
nodes[mask,2]=nodes[mask,2]/dist1[mask]*dist2[mask]
#scale nodes from meter to nano-meter
nodes=nodes*1e-9

nnode=len(nodes)
nelem=len(elements)
nface=len(faces)
print('THE NUMBER OF NODES IS: %d'%nnode)
print('THE NUMBER OF ELEMENTS IS: %d'%nelem)
print('THE NUMBER OF FACES IS: %d'%nface)
print('')

#define computational volume and boundaries
in_active_zones=zones>=3 #True for elements used in computation
with_first_kind_bc=(node_flags==1)|(node_flags==2) #True for nodes on first kind b.c.
with_third_kind_bc=face_flags==20 #True for faces on third kind b.c.

#define electrolyte,solid,stern layer
in_electrolyte=zones>=3
in_solid=zones<=2
in_stern=face_flags==20

# print('Compute distance scaling factor using Debye length')
# debye_len=np.sqrt(perm0*rel_perm_electrolyte*kB*temp_K/2/zval**2/echarge**2/kA/cinf)
# dist_factor=10**(-round(np.log10(debye_len))) #to be determined from debye length
# print('DEBYE LENGTH IS: %.2e m'%debye_len)
# print('DISTANCE SCALING FACTOR IS: %.2e'%dist_factor)
# print('')

print('Saving coefficients to %s'%outfile)
with h5py.File(outfile,'w') as file:
    #group mesh
    file.create_dataset('mesh/nodes',data=nodes,dtype='float')
    file.create_dataset('mesh/elements',data=elements,dtype='int')
    file.create_dataset('mesh/faces',data=faces,dtype='int')
            
    file.create_dataset('mesh/in_active_zones',data=in_active_zones,dtype='bool')
    file.create_dataset('mesh/with_first_kind_bc',data=with_first_kind_bc,dtype='bool')
    file.create_dataset('mesh/with_third_kind_bc',data=with_third_kind_bc,dtype='bool')
    
    file.create_dataset('mesh/in_electrolyte',data=in_electrolyte,dtype='bool')
    file.create_dataset('mesh/in_solid',data=in_solid,dtype='bool')
    file.create_dataset('mesh/in_stern',data=in_stern,dtype='bool')
    #file.create_dataset('mesh/dist_factor',data=dist_factor,dtype='float')
    
    #group physics
    file.create_dataset('physics/rel_perm_electrolyte',data=rel_perm_electrolyte,dtype='float')
    file.create_dataset('physics/rel_perm_solid',data=rel_perm_solid,dtype='float')
    file.create_dataset('physics/diffusion_electrolyte',data=diffusion_electrolyte,dtype='float')
    file.create_dataset('physics/diffusion_stern',data=diffusion_stern,dtype='float')
    file.create_dataset('physics/mobility_electrolyte',data=mobility_electrolyte,dtype='float')
    file.create_dataset('physics/mobility_stern',data=mobility_stern,dtype='float')
    file.create_dataset('physics/sigma_electrolyte',data=sigma_electrolyte,dtype='float')
    file.create_dataset('physics/sigma_stern',data=sigma_stern,dtype='float')

    file.create_dataset('physics/temperature',data=temp_K,dtype='float')
    file.create_dataset('physics/ion_conc_inf',data=cinf,dtype='float')
    file.create_dataset('physics/ion_val',data=zval,dtype='float')
    
    #create subgroups for frequencies
    for i in range(len(freq)):
        file.create_group('frequency_%.6e'%freq[i])
    
print('Done')

Reading sphere.1.node
Reading sphere.1.ele
Reading sphere.1.face
Adjusting mesh by radius (will be removed in the future)
THE NUMBER OF NODES IS: 24157
THE NUMBER OF ELEMENTS IS: 135851
THE NUMBER OF FACES IS: 17578

Saving coefficients to sphere_s61_electrolyte.hdf5
Done


# Mesh for Solid

In [3]:
%reset -f
import h5py,os
import numpy as np

#modify user inputs below
mesh_prefix='sphere'
outfile='sphere_s61_solid.hdf5'
freq=[0.01,0.1] #simulating frequencies

rel_perm_electrolyte=78.5 #relative permittivity of bulk electrolyte [F/m]
rel_perm_solid=4.5 #relative permittivity of solid particle [F/m]
diffusion_electrolyte=2*1e-6**2/1e-3 #diffusion coefficient of water at 20 deg. C is 2*(1e-6*m)^2/(1e-3*s)
diffusion_stern=2*1e-6**2/1e-2 #diffusion coefficient in Stern layer [m^2/s]
mobility_electrolyte=5e-8 #mobility of sodium cation in bulk electrolyte [m^2/(Vs)]
mobility_stern=5e-9 #mobility in Stern layer [m^2/(Vs)]
sigma_electrolyte=-0.01 #surface charge density in electrolyte [C/m^2]
sigma_stern=-0.01 #surface charge density in Stern layer [C/m^2]

temp_K=298 #room/ambient temperature [K]
cinf=100 #ion concentration in bulk electrolyte [mol/m^3]
zval=1 #ion valence in bulk electrolyte
# cinf_pos=0.01 #ion concentration of positive charges
# cinf_neg=0.01 #ion concentration of negative charges
# zval_pos=1 #positive ion valence
# zval_neg=1 #negative ion valence

#set physical constants
echarge=1.602e-19 #elementary charge [C]
perm0=8.85e-12 #vacuum permittivity [F/m]
kA=6.022e23 #Avogadro constant [1/mol]
kB=1.381e-23 #Boltzmann's constant [J/K]

#load mesh
print('Reading %s.1.node'%mesh_prefix)
nodes=np.genfromtxt(mesh_prefix+'.1.node',skip_header=1,skip_footer=1,usecols=(1,2,3))
node_flags=np.genfromtxt(mesh_prefix+'.1.node',skip_header=1,skip_footer=1,usecols=5,dtype='int')

print('Reading %s.1.ele'%mesh_prefix)
elements=np.genfromtxt(mesh_prefix+'.1.ele',skip_header=1,usecols=(1,2,3,4),dtype='int')
zones=np.genfromtxt(mesh_prefix+'.1.ele',skip_header=1,usecols=5,dtype='int')

print('Reading %s.1.face'%mesh_prefix)
faces=np.genfromtxt(mesh_prefix+'.1.face',skip_header=1,usecols=(1,2,3),dtype='int')
face_flags=np.genfromtxt(mesh_prefix+'.1.face',skip_header=1,usecols=4,dtype='int')

#adjust indices to start from zero
elements=elements-1
faces=faces-1

# #translate z coordinate
# print('Translating Z coordinate')
# nodes[:,2]=-(np.power(10,-nodes[:,2]/1000*3)-1) #problem dependent
#adjust mesh by radius
print('Adjusting mesh by radius (will be removed in the future)')
dist1=np.sqrt(nodes[:,0]**2+nodes[:,1]**2+nodes[:,2]**2)
mask=dist1>5.0
dist2=np.zeros_like(dist1)
dist2[mask]=10**((dist1[mask]-5)/50*2)+5-1

# print(min(dist1[mask]),max(dist1[mask]))
# print(min(dist2[mask]),max(dist2[mask]))
nodes[mask,0]=nodes[mask,0]/dist1[mask]*dist2[mask]
nodes[mask,1]=nodes[mask,1]/dist1[mask]*dist2[mask]
nodes[mask,2]=nodes[mask,2]/dist1[mask]*dist2[mask]
#scale nodes from meter to nano-meter
nodes=nodes*1e-9

nnode=len(nodes)
nelem=len(elements)
nface=len(faces)
print('THE NUMBER OF NODES IS: %d'%nnode)
print('THE NUMBER OF ELEMENTS IS: %d'%nelem)
print('THE NUMBER OF FACES IS: %d'%nface)
print('')

#define computational volume and boundaries
in_active_zones=zones<=2 #True for elements used in computation
with_first_kind_bc=(node_flags==1)|(node_flags==2) #True for nodes on first kind b.c.
with_third_kind_bc=face_flags==20 #True for faces on third kind b.c.

#define electrolyte,solid,stern layer
in_electrolyte=zones>=3
in_solid=zones<=2
in_stern=face_flags==20

# print('Compute distance scaling factor using Debye length')
# debye_len=np.sqrt(perm0*rel_perm_electrolyte*kB*temp_K/2/zval**2/echarge**2/kA/cinf)
# dist_factor=10**(-round(np.log10(debye_len))) #to be determined from debye length
# print('DEBYE LENGTH IS: %.2e m'%debye_len)
# print('DISTANCE SCALING FACTOR IS: %.2e'%dist_factor)
# print('')

print('Saving coefficients to %s'%outfile)
with h5py.File(outfile,'w') as file:
    #group mesh
    file.create_dataset('mesh/nodes',data=nodes,dtype='float')
    file.create_dataset('mesh/elements',data=elements,dtype='int')
    file.create_dataset('mesh/faces',data=faces,dtype='int')
            
    file.create_dataset('mesh/in_active_zones',data=in_active_zones,dtype='bool')
    file.create_dataset('mesh/with_first_kind_bc',data=with_first_kind_bc,dtype='bool')
    file.create_dataset('mesh/with_third_kind_bc',data=with_third_kind_bc,dtype='bool')
    
    file.create_dataset('mesh/in_electrolyte',data=in_electrolyte,dtype='bool')
    file.create_dataset('mesh/in_solid',data=in_solid,dtype='bool')
    file.create_dataset('mesh/in_stern',data=in_stern,dtype='bool')
    #file.create_dataset('mesh/dist_factor',data=dist_factor,dtype='float')
    
    #group physics
    file.create_dataset('physics/rel_perm_electrolyte',data=rel_perm_electrolyte,dtype='float')
    file.create_dataset('physics/rel_perm_solid',data=rel_perm_solid,dtype='float')
    file.create_dataset('physics/diffusion_electrolyte',data=diffusion_electrolyte,dtype='float')
    file.create_dataset('physics/diffusion_stern',data=diffusion_stern,dtype='float')
    file.create_dataset('physics/mobility_electrolyte',data=mobility_electrolyte,dtype='float')
    file.create_dataset('physics/mobility_stern',data=mobility_stern,dtype='float')
    file.create_dataset('physics/sigma_electrolyte',data=sigma_electrolyte,dtype='float')
    file.create_dataset('physics/sigma_stern',data=sigma_stern,dtype='float')
    
    file.create_dataset('physics/temperature',data=temp_K,dtype='float')
    file.create_dataset('physics/ion_conc_inf',data=cinf,dtype='float')
    file.create_dataset('physics/ion_val',data=zval,dtype='float')
    
    #create subgroups for frequencies
    for i in range(len(freq)):
        file.create_group('frequency_%.6e'%freq[i])
    
print('Done')

Reading sphere.1.node
Reading sphere.1.ele
Reading sphere.1.face
Adjusting mesh by radius (will be removed in the future)
THE NUMBER OF NODES IS: 24157
THE NUMBER OF ELEMENTS IS: 135851
THE NUMBER OF FACES IS: 17578

Saving coefficients to sphere_s61_solid.hdf5
Done
